<a href="https://colab.research.google.com/github/Praveen76/Loan-Approval-Classifier-Model-using-Non-Linear-Models/blob/main/Loan_Approval_Classifier_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [242]:
# !git clone https://github.com/Praveen76/Loan-Approval-Classifier-Model-using-Non-Linear-Models.git
# %cd Loan-Approval-Classifier-Model-using-Non-Linear-Models

In [243]:
# %pip install shap xgboost plotly scikit-plot seaborn statsmodels tabulate matplotlib imblearn nbformat>=4.2.0


In [244]:

import sys

# Print Python version
print("Python version")
print(sys.version)
print("")

# Print installed package versions
print("Installed package versions")
for module in sys.modules:
    try:
        version = sys.modules[module].__version__
        print(f"{module}: {version}")
    except AttributeError:
        pass


Python version
3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]

Installed package versions
re: 2.2.1
ipykernel._version: 5.5.6
json: 2.0.9
IPython.core.release: 7.34.0
logging: 0.5.1.2
zlib: 1.0
traitlets._version: 5.7.1
traitlets: 5.7.1
socketserver: 0.4
argparse: 1.1
platform: 1.0.8
pygments: 2.16.1
ptyprocess: 0.7.0
pexpect: 4.9.0
decorator: 4.4.2
pickleshare: 0.7.5
backcall: 0.2.0
wcwidth: 0.2.13
prompt_toolkit: 3.0.43
urllib.request: 3.10
IPython: 7.34.0
ipython_genutils._version: 0.2.0
ipython_genutils: 0.2.0
jupyter_client._version: 6.1.12
_ctypes: 1.1.0
ctypes: 1.1.0
zmq.sugar.version: 23.2.1
zmq.sugar: 23.2.1
zmq: 23.2.1
jupyter_core.version: 5.7.1
jupyter_core: 5.7.1
platformdirs.version: 4.2.0
platformdirs: 4.2.0
jupyter_client: 6.1.12
ipykernel: 5.5.6
_curses: b'2.2'
dateutil: 2.8.2
six: 1.16.0
_decimal: 1.70
decimal: 1.70
setuptools._distutils: 3.10.12
_csv: 1.0
csv: 1.0
setuptools.version: 67.7.2
setuptools._vendor.packaging: 23.0
setuptools.extern.packaging: 23.0
setu

In [245]:
import pandas as pd
import numpy as np
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats
from statsmodels.formula.api import ols
import statsmodels.api as sm

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
import sklearn.metrics as metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from tabulate import tabulate

import warnings

warnings.filterwarnings("ignore")

In [246]:
loan_original=pd.read_csv('./data.csv',parse_dates=['DateAdded','ApprovalDate'])
loan_original=loan_original.infer_objects()

loan_original.head()

,LeadID,DateAdded,LeadSourceGroup,LoanOfficer,Group,ZipCode,LoanNumber,ApprovalDate,BranchCode,LoanPurpose,...,BorrowerAge,BorrowerYearsInSchool,BorrowerTotalMonthlyIncome,BorrowerOwnRent,BorrowerCity,BorrowerState,IsCoBorrowerower,CoBorrowerYearsInSchool,CoBorrowerAge,CoBorrowerTotalMonthlyIncome
0,13067,2013-05-01 18:54:00,NaN,"Narmour, Derrick",Refinance Team - 4103,75078.0,41032026021,NaT,4103.0,Refinance ...,...,36,18,6083.00,NaN,Prosper,TX,True,0,0,0.0
1,14396,2013-05-07 16:53:00,TV,"Johnson, Adrienne",Refinance Team - 4103,76064.0,4103198104,2019-07-21,4103.0,Refinance Cash-out ...,...,52,16,8883.71,Own,Maypearl,TX,True,12,55,0.0
2,14855,2013-05-09 12:53:00,NaN,"Frank, Michael",Refinance Team - 4103,76017.0,4102192789,2019-03-26,4102.0,Refinance Cash-out ...,...,60,12,17538.30,Own,Arlington,TX,True,0,0,0.0
3,26269,2013-07-01 17:25:00,TV,"Benitez, Bryan",Refinance Team - 4102,75238.0,4102193097,2019-04-05,4102.0,Refinance Cash-out ...,...,54,16,13299.09,Own,Dallas,TX,True,0,0,0.0
4,37010,2013-08-30 09:58:00,TV,"Nguyen, Tu",Refinance Team - 4102,77450.0,4102190764,2019-02-01,4102.0,Refinance Cash-out ...,...,49,16,6254.91,Own,Katy,TX,True,0,0,0.0


In [247]:
loan_original.columns

Index(['LeadID', 'DateAdded', 'LeadSourceGroup', 'LoanOfficer', 'Group',
       'ZipCode', 'LoanNumber', 'ApprovalDate', 'BranchCode', 'LoanPurpose',
       'LoanType', 'TotalLoanAmount', 'CreditScore', 'CLTV', 'DTI',
       'BorrowerAge', 'BorrowerYearsInSchool', 'BorrowerTotalMonthlyIncome',
       'BorrowerOwnRent', 'BorrowerCity', 'BorrowerState', 'IsCoBorrowerower',
       'CoBorrowerYearsInSchool', 'CoBorrowerAge',
       'CoBorrowerTotalMonthlyIncome'],
      dtype='object')

In [248]:
#Check for duplicate values
duplicate_records = loan_original[loan_original['LoanNumber'].duplicated(keep = False) == True].sort_values(['LoanNumber'])
duplicate_records.to_csv("duplicate_records.csv")

# Duplicate records found. Hence,
loan_original.drop_duplicates(subset='LoanNumber', keep='first', inplace=True)


In [249]:
loan_original[loan_original['LoanNumber'].duplicated(keep = False) == True].sort_values(['LoanNumber'])
loan_original.reset_index(drop=True, inplace=True)

In [250]:
# Filter out wrong BorrowerYearsInSchool records
loan_original = loan_original[loan_original['BorrowerYearsInSchool']>=1]

In [ ]:

#Strip off white space
loan_original.LoanPurpose=[entry.strip() for entry in loan_original.LoanPurpose]
loan_original.LoanType=[entry.strip() for entry in loan_original.LoanType]

In [251]:

#Create Approved Date column
loan_original['Approved']='Yes'
loan_original['Approved'][loan_original.ApprovalDate.isna()]='No'

In [253]:


#Recategorize Loan Purpose column
LP = ['Refinance','Purchase']
loan_original['LoanPurpose']= [v if v in LP else 'Refinance' for v in loan_original.LoanPurpose]

#Calculate difference between current date and DateAdded in days
loan_original['Diff']=pd.Timestamp.now().normalize()-loan_original['DateAdded'][loan_original.ApprovalDate.isna()]
loan_original['Diff']=[entry.days for entry in loan_original['Diff']]

loan_original['Approved']=['No' if ((x>=90) & (y=='Purchase')) else 'Yes' for (x,y) in zip(loan_original.Diff, loan_original.LoanPurpose)]
loan_original['Approved']=['No' if ((x>=45) & (y=='Refinance')) else 'Yes' for (x,y) in zip(loan_original.Diff, loan_original.LoanPurpose)]

#
loan_original['Approved']=loan_original['Approved'].astype('category')
loan_original['IsCoBorrowerower']=loan_original['IsCoBorrowerower'].astype('category')

loan_original.drop('Diff', axis=1, inplace=True)


LP = ['Own','Rent']
loan_original['BorrowerOwnRent']= [v if v in LP else 'Own' for v in loan_original.BorrowerOwnRent]
loan_original['BorrowerOwnRent'][loan_original['BorrowerOwnRent'].isna()]=loan_original['BorrowerOwnRent'].mode().values

loan_original.Approved.value_counts(normalize=True)*100

Yes    73.757682
No     26.242318
Name: Approved, dtype: float64

In [254]:
#Add Borrower & Co-Borrower's income
loan_original['TotalIncome']=loan_original.BorrowerTotalMonthlyIncome + loan_original.CoBorrowerTotalMonthlyIncome
# loan_original.drop(['BorrowerTotalMonthlyIncome','CoBorrowerTotalMonthlyIncome'],axis=1,inplace=True)

#Checck if Total income is 0

loan_original[((loan_original['TotalIncome']<0))]

#Regroup LeadSourceGroup
LSR = ['Internet', 'TV','Radio','Repeat Client' ]
loan_original['LeadSourceGroup'] = [v if v in LSR else 'Other' for v in loan_original.LeadSourceGroup]

zips = ['75', '76', '77', '78', '79']
loan_original['ZipCode'] = [str(zp)[:2] for zp in loan_original['ZipCode']]
loan_original['ZipCode'] = [v if v in zips else 'Other' for v in loan_original.ZipCode]






In [255]:
loan_original.isna().sum()

LeadID                             0
DateAdded                          0
LeadSourceGroup                    0
LoanOfficer                        4
Group                              4
ZipCode                            0
LoanNumber                         0
ApprovalDate                    3683
BranchCode                         2
LoanPurpose                        0
LoanType                           0
TotalLoanAmount                    0
CreditScore                        0
CLTV                               0
DTI                              217
BorrowerAge                        0
BorrowerYearsInSchool              0
BorrowerTotalMonthlyIncome         0
BorrowerOwnRent                    0
BorrowerCity                       2
BorrowerState                      2
IsCoBorrowerower                   0
CoBorrowerYearsInSchool            0
CoBorrowerAge                      0
CoBorrowerTotalMonthlyIncome       0
Approved                           0
TotalIncome                        0
d

In [256]:
#Create bins for Education
# Assuming 'BorrowerYearsInSchool' is a column in the DataFrame
bins = [0, 12, 16, 18, loan_original['BorrowerYearsInSchool'].max()]
group = ['Higher School', 'UnderGrad', 'PostGrad', 'PHD']
loan_original['Education'] = pd.cut(loan_original['BorrowerYearsInSchool'], bins, labels=group)



In [257]:
## Filter out extreme outliers

loan_original=loan_original[loan_original.CLTV<110]
loan_original=loan_original[loan_original.TotalLoanAmount<600000]
loan_original=loan_original[loan_original.CreditScore>550]
loan_original=loan_original[loan_original.TotalIncome<=30000]

In [258]:
#Drop below features
#Drop irrelevant fetures
unused_vars = ['LeadID','DateAdded','LoanOfficer','Group','IsCoBorrowerower','BranchCode',
                    'CoBorrowerTotalMonthlyIncome','BorrowerYearsInSchool','CoBorrowerYearsInSchool','CoBorrowerAge','BorrowerCity',
                    'BorrowerState','Group','ApprovalDate']
loan_original.drop(unused_vars, axis=1, inplace=True)



In [259]:
loan_original.head(2)

,LeadSourceGroup,ZipCode,LoanNumber,LoanPurpose,LoanType,TotalLoanAmount,CreditScore,CLTV,DTI,BorrowerAge,BorrowerTotalMonthlyIncome,BorrowerOwnRent,Approved,TotalIncome,Education
0,Other,75,41032026021,Refinance,Conventional,206500.0,705,60.20,33.0,36,6083.00,Own,No,6083.00,PostGrad
1,TV,76,4103198104,Refinance,Conventional,100000.0,781,54.05,33.0,52,8883.71,Own,Yes,8883.71,UnderGrad


In [260]:
# Map approved categories
loan_original.Approved.replace({'Yes':1,'No':0},inplace=True)

In [261]:
########## Split Features and Target Varible ############

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Assuming 'loan_original' is your DataFrame
# Replace 'loan_original' with the actual name of your DataFrame

# Split original data into train_df and test_df
train_df, test_df = train_test_split(loan_original, test_size=0.2, stratify=loan_original['Approved'], random_state=42)




In [262]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd

class MissValImputer(BaseEstimator, TransformerMixin):
    """Custom imputer for handling missing values in numerical and categorical variables."""

    def __init__(self, numerical_vars: list, categorical_vars: list):
        if not all(isinstance(var, str) for var in numerical_vars + categorical_vars):
            raise ValueError("Variables should be a list of strings.")

        self.numerical_vars = numerical_vars
        self.categorical_vars = categorical_vars

    def fit(self, X: pd.DataFrame, y: pd.Series = None):
        # Store mean and standard deviation for numerical variables
        self.numerical_fill_values = {}

        for var in self.numerical_vars:
            mean_value = X[var].mean()
            std_value = X[var].std()
            self.numerical_fill_values[var] = (mean_value, std_value)

        # Store mode value for categorical variables
        self.categorical_fill_values = {}

        for var in self.categorical_vars:
            mode_value = X[var].mode().values[0]
            self.categorical_fill_values[var] = mode_value

        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        X = X.copy()

        np.random.seed(0)

        # Impute missing values for numerical variables with mean
        for var in self.numerical_vars:
            mean_value, std_value = self.numerical_fill_values[var]

            # Impute missing values using a random distribution
            null_count = X[var].isnull().sum()
            null_random_list = np.random.randint(mean_value - std_value, mean_value + std_value, size=null_count)
            X.loc[np.isnan(X[var]), var] = null_random_list
            X[var] = X[var].astype(X[var].dtype)  # Ensure the correct data type

        # Impute missing values for categorical variables with mode
        for var in self.categorical_vars:
            mode_value = self.categorical_fill_values[var]
            X[var].fillna(mode_value, inplace=True)

        return X

# Example usage:
# Assuming numerical_vars and categorical_vars are lists of numerical and categorical variables
numerical_vars = ['BorrowerAge', 'TotalLoanAmount', 'CreditScore', 'CLTV', 'DTI', 'BorrowerTotalMonthlyIncome']

categorical_vars = ['ZipCode', 'BorrowerOwnRent', 'Education', 'LeadSourceGroup']

# Create an instance of the CustomImputer
miss_val_imputer = MissValImputer(numerical_vars=numerical_vars, categorical_vars=categorical_vars)

# Fit and transform the data
train_df_imputed = miss_val_imputer.fit_transform(train_df)
test_df_imputed = miss_val_imputer.transform(test_df)

# Check the information after imputation
print(train_df_imputed[numerical_vars].isna().sum(), "\n \n")
print(train_df_imputed[categorical_vars].isna().sum(), "\n \n")
print(test_df_imputed[numerical_vars].isna().sum(), "\n \n")
print(test_df_imputed[categorical_vars].isna().sum(), "\n \n")


BorrowerAge                   0
TotalLoanAmount               0
CreditScore                   0
CLTV                          0
DTI                           0
BorrowerTotalMonthlyIncome    0
dtype: int64 
 

ZipCode            0
BorrowerOwnRent    0
Education          0
LeadSourceGroup    0
dtype: int64 
 

BorrowerAge                   0
TotalLoanAmount               0
CreditScore                   0
CLTV                          0
DTI                           0
BorrowerTotalMonthlyIncome    0
dtype: int64 
 

ZipCode            0
BorrowerOwnRent    0
Education          0
LeadSourceGroup    0
dtype: int64 
 



In [263]:
import pandas as pd

class OutlierHandler:
    def __init__(self, numerical_vars: list):
        self.numerical_vars = numerical_vars
        self.outlier_info = {}

    def identify_outliers(self, df, clmn):
        print('Outliers check for feature:', clmn)
        # Outliers check
        Q1 = df[clmn].quantile(0.25)
        Q3 = df[clmn].quantile(0.75)
        IQR = Q3 - Q1
        Lower_Whisker = Q1 - (1.5 * IQR)
        Upper_Whisker = Q3 + (1.5 * IQR)
        print("Q1:", Q1, "Q3:", Q3, "IQR:", IQR, "Lower_Whisker:", Lower_Whisker, "Upper_Whisker:", Upper_Whisker,
              'Range:', (Upper_Whisker - Lower_Whisker))
        # Identify Outliers
        outliers_size = df[(df[clmn] < Lower_Whisker) | (df[clmn] > Upper_Whisker)].shape[0]
        dataset_size = df.shape[0]
        outliers_percentage = round((outliers_size / dataset_size) * 100, 2)
        print('% of outliers:', outliers_percentage, '\n')

        self.outlier_info[clmn] = {'Lower_Whisker': Lower_Whisker, 'Upper_Whisker': Upper_Whisker}

    def treat_outliers(self, df, clmn, method='mean'):
        if method == 'mean':
            mean_value = round(df[clmn].mean(), 2)
            print('Mean value for:', clmn, mean_value)
            df[clmn] = df[clmn].apply(lambda x: mean_value if
                                      (x < self.outlier_info[clmn]['Lower_Whisker'] or x > self.outlier_info[clmn]['Upper_Whisker'])
                                      else x)
        # You can add more methods for treating outliers if needed

    def process_outliers(self, df):
        for col in self.numerical_vars:
            self.identify_outliers(df, col)
            self.treat_outliers(df, col)

# Create an instance of the OutlierHandler
outlier_handler = OutlierHandler(numerical_vars=numerical_vars)

# Process outliers in the training data
outlier_handler.process_outliers(train_df)

# Print the outlier information (if needed)
print(outlier_handler.outlier_info)


Outliers check for feature: BorrowerAge
Q1: 41.0 Q3: 58.0 IQR: 17.0 Lower_Whisker: 15.5 Upper_Whisker: 83.5 Range: 68.0
% of outliers: 0.16 

Mean value for: BorrowerAge 50.19
Outliers check for feature: TotalLoanAmount
Q1: 164000.0 Q3: 315000.0 IQR: 151000.0 Lower_Whisker: -62500.0 Upper_Whisker: 541500.0 Range: 604000.0
% of outliers: 0.08 

Mean value for: TotalLoanAmount 246222.61
Outliers check for feature: CreditScore
Q1: 690.0 Q3: 778.0 IQR: 88.0 Lower_Whisker: 558.0 Upper_Whisker: 910.0 Range: 352.0
% of outliers: 0.13 

Mean value for: CreditScore 731.43
Outliers check for feature: CLTV
Q1: 59.995000000000005 Q3: 80.0 IQR: 20.004999999999995 Lower_Whisker: 29.98750000000001 Upper_Whisker: 110.0075 Range: 80.01999999999998
% of outliers: 1.66 

Mean value for: CLTV 70.19
Outliers check for feature: DTI
Q1: 28.0 Q3: 43.0 IQR: 15.0 Lower_Whisker: 5.5 Upper_Whisker: 65.5 Range: 60.0
% of outliers: 1.25 

Mean value for: DTI 40.38
Outliers check for feature: BorrowerTotalMonthlyInc

In [264]:
from sklearn.preprocessing import OneHotEncoder

class CategoricalEncoder:
    def __init__(self, categorical_vars: list):
        self.categorical_vars = categorical_vars
        self.encoder = None

    def encode(self, df):
        if len(self.categorical_vars) == 0:
            return df, None

        # One-Hot Encoding
        encoder = OneHotEncoder(drop='first', sparse=False)
        OHE = encoder.fit(df[self.categorical_vars])
        cat_feature_names = OHE.get_feature_names_out(self.categorical_vars)

        # Prefix encoded values with column names
        OHE_df = pd.DataFrame(OHE.transform(df[self.categorical_vars]), columns=cat_feature_names)

        num_df = df.drop(self.categorical_vars, axis=1).reset_index(drop=True)

        df_encoded = pd.concat([OHE_df, num_df], axis=1)

        self.encoder = OHE

        return df_encoded

# Create an instance of the CategoricalEncoder
categorical_encoder = CategoricalEncoder(categorical_vars=categorical_vars)

# Encode the training data
train_df = categorical_encoder.encode(train_df)

# Encode the test data
test_df = categorical_encoder.encode(test_df)

# Access the encoder object if needed
OHE = categorical_encoder.encoder


In [265]:
import pickle

pickle.dump(OHE, open('./encoder.pkl' ,'wb'))
enc = pickle.load(open('./encoder.pkl','rb'))



In [266]:
# Split train_df into features (X_train) and target variable (y_train)
X_train = train_df.drop(columns='Approved')
y_train = train_df['Approved']

# Split test_df into features (X_test) and target variable (y_test)
X_test = test_df.drop(columns='Approved')
y_test = test_df['Approved']

# Optionally, convert to DataFrames if needed
X_train = pd.DataFrame(data=X_train, columns=X_train.columns)
X_test = pd.DataFrame(data=X_test, columns=X_test.columns)
y_train = pd.DataFrame(data=y_train, columns=['Approved'])
y_test = pd.DataFrame(data=y_test, columns=['Approved'])

In [267]:
X_train.head()

,ZipCode_76,ZipCode_77,ZipCode_78,ZipCode_79,ZipCode_Other,BorrowerOwnRent_Rent,Education_PHD,Education_PostGrad,Education_UnderGrad,LeadSourceGroup_Other,...,LoanNumber,LoanPurpose,LoanType,TotalLoanAmount,CreditScore,CLTV,DTI,BorrowerAge,BorrowerTotalMonthlyIncome,TotalIncome
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,41052017300,Purchase,FHA,101495.0,697.0,95.00,46.00,54.0,0.00,3166.66
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,41012020011,Refinance,Conventional,333250.0,701.0,59.50,38.00,50.0,9417.00,11945.74
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,41052017547,Purchase,Conventional,510400.0,712.0,89.54,45.00,37.0,10416.68,10416.68
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,4102202659,Refinance,Conventional,140000.0,735.0,67.96,41.00,52.0,5416.67,5416.67
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4103196705,Refinance,Conventional,202500.0,625.0,59.03,40.38,42.0,0.00,4430.13


In [268]:
X_test.head()

,ZipCode_76,ZipCode_77,ZipCode_78,ZipCode_79,ZipCode_Other,BorrowerOwnRent_Rent,Education_PHD,Education_PostGrad,Education_UnderGrad,LeadSourceGroup_Other,...,LoanNumber,LoanPurpose,LoanType,TotalLoanAmount,CreditScore,CLTV,DTI,BorrowerAge,BorrowerTotalMonthlyIncome,TotalIncome
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,4103204927,Refinance,Conventional,223412.0,742,61.71,30.0,47,8336.67,13753.67
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,41012024670,Refinance,VA,377319.0,786,65.62,NaN,55,0.00,0.00
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,41022014100,Refinance,Conventional,321600.0,695,80.00,39.0,36,10523.57,10523.57
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,41021916489,Refinance,Conventional,374696.0,720,85.54,35.0,35,11000.00,11000.00
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,4101208293,Refinance,Conventional,146000.0,682,58.87,43.0,70,5166.91,5166.91


In [269]:
# Standardize Data
scaler = StandardScaler()
X_train[numerical_vars] = scaler.fit_transform(X_train[numerical_vars])
X_test[numerical_vars] = scaler.transform(X_test[numerical_vars])
X_train[numerical_vars].head()

,BorrowerAge,TotalLoanAmount,CreditScore,CLTV,DTI,BorrowerTotalMonthlyIncome
0,0.325416,-1.346867,-0.636156,1.595340,1.155810,-1.925444
1,-0.011641,0.813404,-0.562718,-0.759298,0.333757,0.472903
2,-1.107076,2.464683,-0.360763,1.233190,1.053053,0.727504
3,0.156887,-0.987948,0.061505,-0.198164,0.642027,-0.545912
4,-0.685755,-0.405363,-1.958039,-0.790472,0.578318,-1.925444


In [270]:
X_test[numerical_vars].head()

,BorrowerAge,TotalLoanAmount,CreditScore,CLTV,DTI,BorrowerTotalMonthlyIncome
0,-0.264434,-0.210435,0.190022,-0.612713,-0.488297,0.197762
1,0.409680,1.224187,0.997840,-0.353371,NaN,-1.925444
2,-1.191340,0.704811,-0.672875,0.600422,0.436513,0.754727
3,-1.275604,1.199737,-0.213887,0.967879,0.025487,0.876066
4,1.673643,-0.932020,-0.911548,-0.801084,0.847540,-0.609522


In [271]:
from sklearn.metrics import confusion_matrix ,classification_report
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.model_selection import cross_val_score
import scikitplot as skplt

# Write a function for Grid Search
def GridSearch(model,param_grid,cv,X,y,scoring=None):
    # Random search of parameters
    gcv = GridSearchCV(estimator = model, param_grid=param_grid , refit = True, cv = cv, verbose=1,  n_jobs = -1)
    # Fit the model
    gcv.fit(X,y)

    return gcv.best_params_

def modelPerformance(model,X,y,dataType):
    print("=== Model Performance report on ",dataType," Data ===")
    y_pred = model.predict(X)
    print("=== Confusion Matrix ===")
    print(confusion_matrix(y, y_pred))
    print('\n')
    print("=== Classification Report ===")
    print(classification_report(y, y_pred))
    skplt.metrics.plot_confusion_matrix(y,y_pred, normalize=True, figsize=(8,3))
